# 5. Interval Newton method

Let's go back to thinking about solving equations, i.e. finding *roots* of functions.

In standard (i.e. non-interval) numerical analysis, the [**Newton** (or Newton--Raphson) method] is a powerful method for (sometimes) finding roots, given by the iterative method

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}.$$

This has the geometric interpretation of starting from $(x_n, 0)$, moving vertically until you hit the graph of $f$, then following the **tangent line** at the point $(x_n, f(x_n))$ until it intersects the $x$-axis, which gives the new approximation.

If the initial condition is close enough to a root, then then algorithm converges very fast. *But* it is often very difficult to know when you are "close enough". If you don't start close enough then the algorithm can diverge, oscillate, behave chaotically, etc.

Furthermore, if we would like to find *all* the roots of a given function in a given interval, we are out of luck: we can never know (for a general function) whether there are or are not more roots available.

Interval methods provide a solution to these problems, providing a method that in principle can find and verify
all simple roots of a function in a box.

## Interval Newton method

In [ ]:
using IntervalArithmetic, ForwardDiff

The interval Newton method is an apparently simple modification of the standard Newton method, but which has superpowers! When correctly implemented, it *guarantees* not to miss any roots, and to home in on the roots that are present at a certain speed in 1D. In higher dimensions there similar guarantees.

We do *not* just run the above algorithm using intervals; this will lead to intervals that grow over time and give useless results. Rather we must refine the algorithm to take *advantage* of the fact that we have information over a whole set.

Suppose we start with an initial interval $X_0$; this is the interval in which we wish to find roots of $f$. 

We will partially mimic the standard Newton method by picking a *single* point in the interval to start from; a standard, but not obligatory, choice is the midpoint $m := \text{mid}(X)$, sometimes denoted $\check{X}$.

We move up to the graph at $(m, f(m))$. But now instead of picking the tangent line, we pick *all possible directions simultaneously*, where the direction is determined by *a possible value of the derivative $f'(x)$*.
An operational version of this is:

$$N_f(X) := m(X) - \frac{f(m(X))}{f'(X)}$$

Note that we are calculating an enclosure of the derivative over the *whole* interval; this will contain all possible derivatives over the interval. 

It turns out that the mean-value theorem now tells us that *any root of $f$ must be hit by one of these sloped lines*. Note that intervals are enabling us to make the mean-value theorem into a *constructive* method.

[Technical note: To bound rounding errors, $m(X)$ must be treated as an interval with 0 width when calculating $f(m)$.]

## Using the interval Newton operator

The following two theorems can be shown:

- If $N_f(X) \subset X$ then there is a unique zero of $f$ in $X$ (by a contraction argument).

- If $N_f(X) \cap X = \emptyset$ then there are no roots of $f$ in $X$ (by the above argument using the mean-value theorem).

#### Exercise

1. Implement this and try some examples.


2. Once you have found a box $X$ in which you have located a root, iterate the Newton operator (i.e. apply it repeatedly) to that box. It will shrink down quadratically fast to a tiny interval containing the root!

## Division by zero

Often, $f'$ will hit zero somewhere in $X$. In this case the division seems not to make any sense, or will return $[-\infty, \infty]$.

But in fact there will always be a region which is excluded from $f'(X)$ near $\pm \infty$, which will lead to an excluded region around $0$ in $1 / f'(X)$. We cannot represent this with a single interval, but we can using the union of *two* intervals.

#### Exercise

1. Implement this "extended division".

## Higher dimensions

The Newton interval method extends to functions $f: \mathbb{R}^n \to \mathbb{R}^n$, but now the derivative becomes the Jacobian matrix $\mathsf{J}$, and we replace $1 / f'(X)$ with the matrix inverse, giving

$$N_f(X) := m(X) - \mathsf{J}(m(X))^{-1} f(m(X))$$

The same theorem holds as stated for 1D above on the uniqueness or absence of roots in a given box!

To calculate this numerically we use Julia's `\` operator from the `LinearAlgebra` standard library to solve the  linear system of interval equations. This will give as an over-approximation of the result.

[This usually works, but we have had some issues, in particular when using `StaticArrays.jl`. We are still trying to iron this out. There are many techniques to solve linear systems of interval equations.]

#### Exercise

1. Implement this. Use it to locate a root of

    $g(x, y) = (x^2 + y^2 - 1, x + y)$.
    
    Hint: You will need to start close enough to a root.